In [1]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import MultipleLocator

from curvelops import FDCT2D
from curvelops.plot import create_colorbar

from PIL import Image, ImageOps

# Load the image and simulate random sampling

In [2]:
def showimage(I):
    A = Image.fromarray(np.uint8(I)*255)
    return A 

In [3]:
# load the image
im = Image.open('lena_std.tif')
im = ImageOps.grayscale(im)
I = np.asarray(im)

In [4]:
# simulate the sampling and get the sampling 
from random import sample
import math

# generate the sampling matrix
pctg = 0.01
arrsize = im.size[1]*im.size[0]
arr = np.arange(0,arrsize-1)
kx,ky = (np.random.choice(arr,round(pctg*arrsize)), np.random.choice(arr,round(pctg*arrsize)))

# 
kx = kx % I.shape[0]
ky = ky % I.shape[1]
k = np.array([kx,ky])
k = np.reshape(k,(-1,2))

Y = np.zeros((I.shape))
M = np.zeros((I.shape))
sampled = np.zeros((k.shape[0]))
for i in range(0,k.shape[0]):
    sampled[i] = I[k[i,0],k[i,1]]
    Y[k[i,0],k[i,1]] = I[k[i,0],k[i,1]]
    M[k[i,0],k[i,1]] = 1

# Try Scattered Interpolation

In [5]:
def scattered_interpolation(k,sampled):
    from scipy.interpolate import griddata
    # create grid
    grid_x,grid_y = np.meshgrid(np.arange(0,im.size[0]-1),np.arange(0,im.size[1]-1),indexing='ij')
    
    itpd = griddata(k,sampled.astype('double'),(grid_x,grid_y),method='nearest')
    return itpd

X = scattered_interpolation(k,sampled)

# CS-CVT from Matlab

We do have
- $Y$ for observable
- $M$ for mask
- $X$ for initial point

### Try to compute the Curvelet Coeff

In [8]:
C = FDCT2D(
    X.shape,
    allcurvelets=True)

C_fdct_struct = C.struct(C @ X)